In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse

hparams = {
    'npratio':4,
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:0',
    'attrs': ['title'],
    'k': None
}

device = torch.device(hparams['device'])

## show data

In [ ]:
_, loader_train, loader_test = prepare(hparams)

In [ ]:
a = next(iter(loader_train))
b = next(iter(loader_test))
a

In [ ]:
a['candidate_title']

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [3]:
hparams['scale'] = 'large'
hparams['mode'] = 'tra'
analyse(hparams)

avg_title_length:10.67731736385395
 avg_abstract_length:36.4448570331045
 avg_his_length:32.99787212887438
 avg_impr_length:37.40116394684935
 cnt_his_lg_50:447829
 cnt_his_eq_0:46065
 cnt_imp_multi:567571


### tailor Data to demo size

In [ ]:
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)